In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from statsmodels.tsa.stattools import coint
import seaborn as sns
import winsound
import math
from statistics import mean
import pickle
from tabulate import tabulate

In [2]:
df = pd.read_csv("Prices.csv")
df.drop(["Name"],axis=1,inplace=True)
df.drop(["PX_VOLUME"],axis=1,inplace=True)
df['date']= pd.to_datetime(df['date'],dayfirst=True)

In [3]:
pickle_in = open("dict.pickle","rb")
pairs_from_dict = pickle.load(pickle_in)
pairs_from_dict

{Timestamp('2008-01-01 00:00:00'): [['EIM IS Equity', 'RIL IS Equity'],
  ['EIM IS Equity', 'ITC IS Equity'],
  ['EIM IS Equity', 'TTAN IS Equity'],
  ['EIM IS Equity', 'APNT IS Equity'],
  ['EIM IS Equity', 'HDFC IS Equity'],
  ['EIM IS Equity', 'BHARTI IS Equity'],
  ['EIM IS Equity', 'ADSEZ IS Equity'],
  ['EIM IS Equity', 'TTMT IS Equity'],
  ['EIM IS Equity', 'UPLL IS Equity'],
  ['AXSB IS Equity', 'KMB IS Equity'],
  ['AXSB IS Equity', 'LT IS Equity'],
  ['AXSB IS Equity', 'ICICIBC IS Equity'],
  ['AXSB IS Equity', 'ADSEZ IS Equity'],
  ['AXSB IS Equity', 'GRASIM IS Equity'],
  ['RIL IS Equity', 'JSTL IS Equity'],
  ['GAIL IS Equity', 'APNT IS Equity'],
  ['GAIL IS Equity', 'ONGC IS Equity'],
  ['ITC IS Equity', 'BHARTI IS Equity'],
  ['ITC IS Equity', 'ADSEZ IS Equity'],
  ['MM IS Equity', 'RIL IS Equity'],
  ['MM IS Equity', 'Z IS Equity'],
  ['MM IS Equity', 'HDFC IS Equity'],
  ['HNDL IS Equity', 'TCS IS Equity'],
  ['TATA IS Equity', 'TCS IS Equity'],
  ['Z IS Equity', 'LT I

Making date list having month's starting date starting from 2008 Jan to 2019 Nov 

In [89]:
start_date=pd.date_range('2008-01-01','2019-11-30', freq='MS').strftime("%Y-%b-%d").tolist()
df['date']= pd.to_datetime(df['date'],dayfirst=True)
start_date = pd.to_datetime(start_date,dayfirst=True)

Function for calculating returns given a pair, start and end date of the trading period. Also the return over the invested amount.

In [9]:
def returns (pair, st_date,end_date,amount):
    y=amount
    df1=df.copy()
    df1 = df1[df1['Ticker']==pair[0]]
    df1 = df1[df1['date']>=st_date]
    df1 = df1[df1['date']<end_date]
    df2=df.copy()
    df2 = df2[df2['Ticker']==pair[1]]
    df2 = df2[df2['date']>=st_date]
    df2 = df2[df2['date']<end_date]
    date = df1['date'].tolist()
    pr1 = df1['PX_LAST'].tolist()
    pr2 = df2['PX_LAST'].tolist()

    main_df = pd.DataFrame(
        {'date': date,
         pair[0]: pr1,
         pair[1]: pr2})
    
    ratio = []
    for i in range(len(main_df)):
        ratio.append(math.log(main_df.iloc[i][pair[1]]/main_df.iloc[i][pair[0]]))
    main_df['ln']=ratio
    
    
    mvg_avg=[]
    std_dev=[]
    for i in range(len(main_df)):
        window=[]
        if i<10:
            mvg_avg.append(0)
        #mvg_avg.append(0)
            std_dev.append(0)
        #std_dev.append(0)
        else:
            for j in range(0,10):
                window.append(main_df.iloc[i-j-1]['ln'])
            #last10.append(df2.iloc[i-j-1]['ratio'])
            mvg_avg.append(mean(window))
        #mvg_avg.append(mean(last10))
            std_dev.append(np.std(window))
        #std_dev.append(np.std(last10))

    main_df['mvg_avg']=mvg_avg
    main_df['std_dev']=std_dev
    
    z_score=[]
    #z_score=[]
    for i in range(len(main_df)):
        if i<10:
            z_score.append(0)
#        z_score.append(0)
        else:
            z_score.append((main_df.iloc[i]['ln']-main_df.iloc[i]['mvg_avg'])/main_df.iloc[i]['std_dev'])
#        z_score.append((df2.iloc[i]['ratio']-df2.iloc[i]['mvg_avg'])/df2.iloc[i]['std_dev'])
    main_df['z_score']=z_score
    
    buy_threshold=-2
    sell_threshold=2
    SL=4
    TP=0
    
    signal=[]
#signal=[]

    for i in range(len(main_df)):
        if (main_df.iloc[i]['z_score']<buy_threshold and abs(main_df.iloc[i]['z_score'])<SL):
            signal.append('BUY')
        elif (main_df.iloc[i]['z_score']>sell_threshold and abs(main_df.iloc[i]['z_score'])<SL):
            signal.append('SELL')
        else:
            signal.append('')
    main_df['signal']=signal
    
    
    status=['']*len(main_df)
    for i in range (len(main_df)):
        #print (i)
        if status[i-1]=='' or status[i-1]=='SL' or status[i-1]=='TP':
            status[i]= main_df.iloc[i]['signal']
            #print (status[i])
        elif abs(z_score[i])> SL:
            if status[i-1]=='BUY' or status[i-1]=='SELL':
                status[i]= 'SL'
                #print (status[i])
        elif z_score[i-1]/z_score[i]<0:
            if status[i-1]=='BUY' or status[i-1]=='SELL':
                status[i]= 'TP'
                #print (status[i])
        else:
            status[i]=status[i-1]
            #print (status[i])
            
    main_df['status']=status
    pos1_open=0
    pos1_close=0
    pos2_open=0
    pos2_close=0
    position=0
    profit=[]
    pos1_pnl=0
    pos2_pnl=0
            
    for i in range (len(main_df)):
        
        if pos1_open==0:
            #print(i)
            #print (pos1_open)
            if status[i]=='BUY':
                pos1_open=main_df.iloc[i][pair[1]]
                pos2_open=main_df.iloc[i][pair[0]]
                position = 1
                #print (pos1_open)
            if status[i]=='SELL':
                pos1_open=main_df.iloc[i][pair[1]]
                pos2_open=main_df.iloc[i][pair[0]]
                position = -1
                #print (pos1_open)
        if status[i]=='SL' or status[i]=='TP':
            #print (i)
            pos1_close= main_df.iloc[i][pair[1]]
            pos2_close= main_df.iloc[i][pair[0]]
            if position ==-1:
                pos1_pnl=(pos1_open-pos1_close)*100/pos1_open
                pos2_pnl=(pos2_close-pos2_open)*100/pos2_open
            if position ==1:
                pos2_pnl=(pos2_open-pos2_close)*100/pos2_open
                pos1_pnl=(pos1_close-pos1_open)*100/pos1_open
            pos1_open=0
            pos1_close=0
            pos2_open=0
            pos2_close=0
            position=0
            profit.append(pos1_pnl+pos2_pnl)
    #print (position)
    #print (pos1_open)
    #print (pos1_close)
    #print (pos2_open)
    #print (pos2_close)
    if position!=0:
        main_df.at[len(main_df)-1, 'status']='Square Off'
        pos1_close=main_df.iloc[len(main_df)-1][pair[1]]
        pos2_close=main_df.iloc[len(main_df)-1][pair[0]]
        if position==1:
            pos2_pnl=(pos2_open-pos2_close)*100/pos2_open
            pos1_pnl=(pos1_close-pos1_open)*100/pos1_open
        if position ==-1:
            pos1_pnl=(pos1_open-pos1_close)*100/pos1_open
            pos2_pnl=(pos2_close-pos2_open)*100/pos2_open
        #print (pos1_close)
        #print (pos2_close)
        pos1_open=0
        pos1_close=0
        pos2_open=0
        pos2_close=0
        position=0
        profit.append(pos1_pnl+pos2_pnl)
        
        
    for i in range (len(profit)):
        y = y*(1+profit[i]/100)
        #print (y)
    return sum(profit), profit, main_df, y
    
    

In [57]:
st_date = datetime.datetime(2019, 2, 1)
end_date = datetime.datetime(2019, 5, 1)
invested_amount = 1000
pair = ['RIL IS Equity', 'INFO IS Equity']
net_profit, profit_by_trades, history, amount=returns(pair, st_date,end_date,invested_amount)
trading_sheet = history.copy()
trading_sheet.drop(['ln','mvg_avg','std_dev'],inplace=True,axis=1)
trading_sheet.set_index('date',inplace=True)
#trading_sheet

Printing the individual pairs trade results

In [58]:
print('The pair involves\033[1m',pair[0],'\033[0mand\033[1m',pair[1],'\033[0mover the trading period\033[1m',st_date,'\033[0m to \033[1m',end_date,'\033[0m')
print('The net profit over the trading period is\033[1m',round(net_profit,ndigits=2),'%\033[0m')
print('If\033[1m',round(invested_amount,ndigits=2),'\033[0mINR is invested over the trading period (with compounding), it finally amounts to\033[1m',round(amount,ndigits=2),'\033[0m INR')
print('The respective profit on each individual position is\033[1m',list(np.around(np.array(profit_by_trades),2)),'\033[0m in %')
print('The trading sheet over the period is:-')
print(tabulate(trading_sheet, headers='keys', tablefmt='psql'))

The pair involves RIL IS Equity and INFO IS Equity over the trading period 2019-02-01 00:00:00  to  2019-05-01 00:00:00 
The net profit over the trading period is 5.41 %
If 1000 INR is invested over the trading period (with compounding), it finally amounts to 1052.47  INR
The respective profit on each individual position is [4.1, -4.64, 1.66, 4.29]  in %
The trading sheet over the period is:-
+---------------------+-----------------+------------------+-------------+----------+----------+
| date                |   RIL IS Equity |   INFO IS Equity |     z_score | signal   | status   |
|---------------------+-----------------+------------------+-------------+----------+----------|
| 2019-02-01 00:00:00 |         1249.95 |           757.05 |  0          |          |          |
| 2019-02-04 00:00:00 |         1290.9  |           755.9  |  0          |          |          |
| 2019-02-05 00:00:00 |         1291.55 |           754.85 |  0          |          |          |
| 2019-02-06 00:00:00 

function for calculating portfolio of returns(equal weighted portfolio)

In [108]:
def portfolio_returns (st_date,end_date,amount):
    #pair_date = datetime.datetime()
    pair_list = []
    #print(type(start_date))
    for i in range(len(start_date)):
        if st_date==start_date[i]:
            #print(st_date)
            #print(start_date[i])
            pair_date = start_date[i-12]
            #print(pair_date)
    for time in pairs_from_dict:
        if pair_date==time:
            pair_list=pairs_from_dict[time]
    return_list=[]
    final_amount_list = []
    no_pairs=0
    for pair in pair_list:
        no_pairs = no_pairs+1
        net_profit, profit_by_trades, history, final_amount = returns(pair, st_date,end_date,invested_amount)
        return_list.append(net_profit)
        final_amount_list.append(final_amount)
    portfolio_return = sum(return_list)
    total_amount = sum(final_amount_list)
    
    return portfolio_return, total_amount, no_pairs

In [117]:
st_date = datetime.datetime(2018, 9, 1)
end_date = datetime.datetime(2018, 12, 1)
amount_invested=1000
per_portfolio_return, final_amount, no_pairs  = portfolio_returns (st_date,end_date,amount_invested)

In [118]:
print(round(per_portfolio_return,2),round((final_amount/(1000*no_pairs)-1)*100,2))

74.79 6.06
